# FL Spreadsheet Creator - Demo

This notebook demonstrates how to use the **FL Spreadsheet Creator** to generate and update a translation spreadsheet from pairs of English and FL text.

Please note: You must come with the English and FL text already prepared. The inputs must have the same number of words for the script to work correctly.

Additionally, the input texts are aligned, apostrophes and dashes have been removed, and the specific input will depend on the particular FL language you are working with. This example uses the Aylid language.

In [2]:
# Import necessary libraries
import pandas as pd
import re
import os
from IPython.display import display

In [3]:
def update_translation_spreadsheet(english_text, fl_text, output_file):
    # Clean and split the texts
    english_words = re.findall(r'\b\w+\b', english_text.lower())
    fl_words = re.findall(r'\b\w+\b', fl_text.lower())
    
    # Check if the number of words match
    if len(english_words) != len(fl_words):
        print(f"\nError: Word count mismatch!")
        print(f"English words ({len(english_words)}): {', '.join(english_words)}")
        print(f"FL words ({len(fl_words)}): {', '.join(fl_words)}")
        print("\nPlease make sure the texts have the same number of words before proceeding.")
        return
    
    # Create word pairs
    word_pairs = list(zip(english_words, fl_words))
    
    try:
        # Try to read existing Excel file
        df = pd.read_excel(output_file)
    except FileNotFoundError:
        # Create new DataFrame if file doesn't exist
        df = pd.DataFrame(columns=['English', 'FL', 'FL_Alt', 'English_Alt'])
    
    alternatives_added = []
    
    # Create dictionaries to store all translations
    eng_to_fl_dict = {}
    fl_to_eng_dict = {}
    
    # First, load existing translations into dictionaries
    for _, row in df.iterrows():
        eng = row['English']
        fl = row['FL']
        
        if eng not in eng_to_fl_dict:
            eng_to_fl_dict[eng] = set()
        eng_to_fl_dict[eng].add(fl)
        
        # Add alternatives if they exist
        if pd.notna(row['FL_Alt']):
            eng_to_fl_dict[eng].update(row['FL_Alt'].split(', '))
            
        if fl not in fl_to_eng_dict:
            fl_to_eng_dict[fl] = set()
        fl_to_eng_dict[fl].add(eng)
        
        # Add alternatives if they exist
        if pd.notna(row['English_Alt']):
            fl_to_eng_dict[fl].update(row['English_Alt'].split(', '))
    
    # Add new word pairs to dictionaries
    for eng, fl in word_pairs:
        if eng not in eng_to_fl_dict:
            eng_to_fl_dict[eng] = set()
        eng_to_fl_dict[eng].add(fl)
        
        if fl not in fl_to_eng_dict:
            fl_to_eng_dict[fl] = set()
        fl_to_eng_dict[fl].add(eng)
    
    # Create the final DataFrame with alternates
    final_data = []
    for eng, fl_set in eng_to_fl_dict.items():
        fl_list = sorted(list(fl_set))
        primary_fl = fl_list[0]
        alt_fl = ', '.join(fl_list[1:]) if len(fl_list) > 1 else ''
        
        # Get English alternatives for the primary FL
        eng_alts = sorted(list(fl_to_eng_dict[primary_fl] - {eng}))
        eng_alt_str = ', '.join(eng_alts) if eng_alts else ''
        
        final_data.append({
            'English': eng,
            'FL': primary_fl,
            'FL_Alt': alt_fl,
            'English_Alt': eng_alt_str
        })
    
    # Create final DataFrame and sort by English
    result_df = pd.DataFrame(final_data).sort_values('English').reset_index(drop=True)
    
    # Save the file
    result_df.to_excel(output_file, index=False)
    
    # Print conflicts if they exist
    conflicts = result_df[(result_df['FL_Alt'] != '') | (result_df['English_Alt'] != '')]
    if not conflicts.empty:
        print("\nWords with multiple translations found:")
        print(conflicts[['English', 'FL', 'FL_Alt', 'English_Alt']])
    
    print(f"\nSpreadsheet updated successfully: {output_file}")

def view_translation_file(file_path):
    if os.path.exists(file_path):
        df = pd.read_excel(file_path)
        print("\nCurrent contents of the translation file:")
        print(df.to_string())
        print(f"\nTotal entries: {len(df)}")
    else:
        print(f"File not found: {file_path}")

## Example Usage

Below, we will use the FL **Spreadsheet Creator** to generate or update a translation spreadsheet. The user must provide the English and FL text as inputs.

In [4]:
# Input texts
english_text = "The output power of modern day lasers ranges from to megawatts in cases where they deliver the output power."
fl_text = "Fad yrisyr gwang nayn tiov ered ashe uvayn eno kij edrylaeth neste selig rano keru angeskar fad yrisyr gwang."

# Output file
output_file = 'AYLIDdemo.xlsx'

# Run the spreadsheet update function
update_translation_spreadsheet(english_text, fl_text, output_file)


Spreadsheet updated successfully: AYLIDdemo.xlsx


In [5]:
# View the translation file to confirm it was updated
view_translation_file(output_file)


Current contents of the translation file:
      English         FL  FL_Alt  English_Alt
0       cases      selig     NaN          NaN
1         day       ered     NaN          NaN
2     deliver   angeskar     NaN          NaN
3        from        eno     NaN          NaN
4          in      neste     NaN          NaN
5      lasers       ashe     NaN          NaN
6   megawatts  edrylaeth     NaN          NaN
7      modern       tiov     NaN          NaN
8          of       nayn     NaN          NaN
9      output     yrisyr     NaN          NaN
10      power      gwang     NaN          NaN
11     ranges      uvayn     NaN          NaN
12        the        fad     NaN          NaN
13       they       keru     NaN          NaN
14         to        kij     NaN          NaN
15      where       rano     NaN          NaN

Total entries: 16


## Conclusion

This notebook demonstrates how to use the FL Spreadsheet Creator to maintain and update translation records between English and the FL language.

Make sure that your inputs are well-prepared and that the number of words matches between the two languages.